In [1]:
from DataPreprocessor import ProcessData
train_path = '../data/Train_Data.csv'
test_path = '../data/Test_Data.csv'

In [2]:
max_sent_len = 100
process_data = ProcessData(train_path, test_path, max_sent_len)
train_data, train_pos_data, train_label = process_data.get_train_data()
test_data, test_pos_data = process_data.get_test_data()

数据预处理...


In [3]:
from ModelSet import NERModel
from GetData import GetData

/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
import pickle

obj = {}
obj['train_data'] = train_data
obj['train_pos_data'] = train_pos_data
obj['train_label'] = train_label
obj['test_data'] = test_data
obj['test_pos_data'] = test_pos_data
obj['char2id'] = process_data.char2id
obj['chunk2id'] = process_data.chunk2id
obj['id2chunk'] = process_data.id2chunk
def save_obj(obj ):
    with open('../data/save_data.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
save_obj(obj)
def load_obj():
    with open('../data/save_data.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
dic = load_obj()
train_data, train_pos_data = dic.get('train_data'), dic.get('train_pos_data')
train_label, test_data = dic.get('train_label'), dic.get('test_data')
test_pos_data, char2id = dic.get('test_pos_data'), dic.get('char2id')
chunk2id, id2chunk = dic.get('chunk2id'), dic.get('id2chunk')

In [8]:
max_sent_len = 0
for item in train_data:
    if len(item) > max_sent_len:
        max_sent_len = len(item)
for item in test_data:
    if len(item) > max_sent_len:
        max_sent_len = len(item)

In [9]:
ner = NERModel(maxlen = 100, word_dict_size = len(char2id), pos_dict_size = 57, word_vec_size = 128, class_label_count = len(chunk2id))
# ner.train2(train_data, train_label )


W0830 18:51:04.466866 4553582016 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0830 18:51:04.480997 4553582016 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0830 18:51:04.491507 4553582016 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0830 18:51:05.032935 4553582016 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_w

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 100, 128)     746496      input_layer[0][0]                
__________________________________________________________________________________________________
zero_padding1d_1 (ZeroPadding1D (None, 104, 128)     0           embedding_layer[0][0]            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 100, 32)      20512       zero_padding1d_1[0][0]           
__________________________________________________________________________________________________
bidirectio

In [10]:
ner_result = ner.predict2(test_data, id2chunk)

W0830 18:51:08.854342 4553582016 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [11]:
import pandas as pd
test_table = pd.read_csv(test_path)

In [23]:
test_table.tail()

,id,title,text
4993,b48976d6,,培育发展农科小额贷款公司、农科农业保险公司、农科融资租赁公司等新型农村金融主体，创新担保模式
4994,c38e4640,,关于外汇交易平台，目前市场面临ZC的高压，一些平台不是退出市场就是跑路，就连人气平台艾拓思也...
4995,53315bd1,Finci芬吉不正规！老师带着我做却血本无归怎么办？,铭笙法援团队V：ms18571483376承诺不追回不收费 前言：Finci芬吉是否正...
4996,24366b47,,爱汇宝APP，一个骗子公司，骗了我十几万，现在跑路了，大家不
4997,3b453440,机会！大荔首批赶街团长开始招募了，月入过万不是梦，会微信就能免费申请,{IMG:1}北上广生活节奏那么紧张，为什么还不赶紧逃离？咱小县城生活这么悠闲悠哉，为什么还...


In [13]:
test_table = test_table.fillna('')
data = []
for _, row in test_table.iterrows():
    content = row['title']+row['text']
    data.append(content)

In [14]:
def get_entity_list(words_list, chunk_list):
    candidate_a = []
    a_list = []
    for i in range(len(words_list)):
        if chunk_list[i] == 'B':
            if len(candidate_a) == 0:
                candidate_a.append(words_list[i])  
            elif len(candidate_a) == 1:
                candidate_a = []
                candidate_a.append(words_list[i]) 
            else:
                a_list.append(''.join(candidate_a))
                candidate_a = []
                candidate_a.append(words_list[i]) 
        elif chunk_list[i] == 'I' and len(candidate_a)!=0 and chunk_list[i-len(candidate_a)] == 'B':
            candidate_a.append(words_list[i])
          
        else:
            if len(candidate_a) != 0:
                a_list.append(''.join(candidate_a))    
            candidate_a = []
    if len(candidate_a) != 0:
        a_list.append(''.join(candidate_a))
    return a_list

In [15]:
result = []
for i in range(len(test_data)):
    data = test_data[i]
    data = [process_data.id2char.get(w) for w in data]
    data = ''.join(data)
    a = get_entity_list(data, ner_result[i])
    result.append(';'.join(a))

In [16]:
ret = pd.DataFrame()
ret['id'] = process_data.test_id
ret['unknownEntities'] = result

In [43]:
ret.head()

,id,unknownEntities
0,83dcefb7,时空周转
1,83dcefb7,时空周转
2,83dcefb7,时空周转
3,83dcefb7,时空周转
4,83dcefb7,


In [44]:
new_id = []
new_result = []
groups = ret.groupby('id')
for _, g in groups:
    new_id.append(_)
    r = list(g['unknownEntities'])
    temp =set()
    for item in r:
        if item.strip() != '':
            hadflag = False
            for had in temp:
                if had in item or item in had:
                    hadflag = True
            if not hadflag:
                temp.add(item)
    
    new_result.append(';'.join(list(set(temp))))      
    

In [45]:
nret = pd.DataFrame()
nret['id'] = new_id
nret['unknownEntities'] = new_result

In [37]:
nret['unknownEntities'] = nret['unknownEntities'].apply(lambda x: ';'.join(set(str(x).split(';'))))

In [38]:
unknownEntities = list(nret['unknownEntities'])
new = []
for item in unknownEntities:
    
    item = item.split(';')
    item = list(set(item))
    item = [i for i in item if len(i.strip())>1]
    new.append(';'.join(item))
    
nret['unknownEntities'] = new

In [40]:
nret.head()

,id,unknownEntities
0,00081566,中海选;熊猫;火币;比特币
1,000b8b75,微信;宝贝;嗨团问;嗨团产;嗨团赚;宝爸;嗨团创;百分百;嗨团分;嗨团有;嗨团平;嗨团总;嗨...
2,0012d20a,云纪;杭州;首AI;思路;人工智
3,001ac252,淘宝天
4,0033ebe3,琼）;弘高融;悦生活;金融;惠民生;车享贷


In [46]:
nret.to_csv('../result/2019-08-40.csv', index=False)